In [1]:
# default_exp file_attention

In [32]:
# export
import os
import ast
import tqdm
import json
import attr
from operator import itemgetter

from scarce_learn import zero_shot
from mlutil.feature_extraction import embeddings
import itertools


import pandas as pd
import numpy as np
from sklearn import feature_extraction, metrics, model_selection

import matplotlib.pyplot as plt
import gensim

from github_search import paperswithcode_tasks

import mlutil
from functools import partial


from scarce_learn.zero_shot import devise_jax, devise_torch

In [33]:
import itertools
import seaborn as sns
import livelossplot

In [3]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import copy

import torch
from torch import nn
from torch.nn import functional as F

from sklearn import model_selection
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset

from transformers import DataCollatorForLanguageModeling
import tqdm
from transformers import RobertaForCausalLM, RobertaModel
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
%cd ..

/home/kuba/Projects/github_search


In [5]:
all_functions_df = pd.read_csv(
    "output/python_functions.csv", index_col="Unnamed: 0", nrows=500000
)

In [6]:
functions_df = all_functions_df.iloc[:500000]

In [7]:
functions_df = functions_df[
    ~functions_df["repo_name"].isin(
        [
            "tensorflow/models",
            "google-research/google-research",
            "tensorflow/tensor2tensor",
            "yumoh/catboost_iter",
        ]
    )
]

In [8]:
repo_functions_count = functions_df["repo_name"].value_counts()

In [9]:
functions_df = functions_df[
    functions_df["repo_name"].isin(
        repo_functions_count[repo_functions_count > 10].index
    )
]

In [10]:
import ast

In [11]:
paperswithcode_with_imports_df = pd.read_csv("output/papers_with_imports.csv")
paperswithcode_with_imports_df["tasks"] = paperswithcode_with_imports_df["tasks"].apply(
    ast.literal_eval
)

In [12]:
functions_df

,repo_name,path,function_name,function_code
0,trangvu/ape-npi,translate/import_graph.py,ImportGraph,class ImportGraph():\n ' Importing and run...
1,trangvu/ape-npi,translate/import_graph.py,load_checkpoint,"def load_checkpoint(sess, checkpoint_dir, file..."
2,trangvu/ape-npi,translate/models.py,auto_reuse,"def auto_reuse(fun):\n ""\n Wrapper that ..."
3,trangvu/ape-npi,translate/models.py,CellWrapper,class CellWrapper(RNNCell):\n '\n Wrappe...
4,trangvu/ape-npi,translate/models.py,multi_encoder,"def multi_encoder(encoder_inputs, encoders, en..."
...,...,...,...,...
499995,huggingface/transformers,tests/test_modeling_deit.py,DeiTModelIntegrationTest,@require_vision\nclass DeiTModelIntegrationTes...
499996,huggingface/transformers,tests/test_feature_extraction_detr.py,DetrFeatureExtractionTester,class DetrFeatureExtractionTester(unittest.Tes...
499997,huggingface/transformers,tests/test_feature_extraction_detr.py,DetrFeatureExtractionTest,@require_torch\n@require_vision\nclass DetrFea...
499998,huggingface/transformers,tests/test_processor_clip.py,CLIPProcessorTest,@require_vision\nclass CLIPProcessorTest(unitt...


In [13]:
functions_df.groupby("repo_name").agg("count")["path"].value_counts()

13      23
15      22
12      21
14      18
21      18
        ..
1200     1
743      1
406      1
531      1
528      1
Name: path, Length: 246, dtype: int64

In [14]:
import sentence_transformers.models

import sentence_transformers

In [15]:
model_name = "microsoft/codebert-base"

In [16]:
sentence_transformer_model = sentence_transformers.SentenceTransformer(model_name)

device = 'cuda' if torch.cuda.is_available() else 'cpu' # Tell pytorch to run this model on the GPU.
tokenizer = RobertaTokenizer.from_pretrained(model_name,truncation = True)
model = RobertaModel.from_pretrained(
    model_name,
    output_attentions = False, 
    output_hidden_states = True,
)
model = model.to(device).half()


path_tokenizer = RobertaTokenizer.from_pretrained(model_name,truncation = True)
path_model = RobertaForMaskedLM.from_pretrained(
    model_name,
    output_attentions = False, 
    output_hidden_states = True,
).roberta
path_model = model.to(device).half()


In [17]:
function_paths = functions_df["repo_name"] + " " + functions_df["path"]

In [18]:
functions_grouped = functions_df.groupby(function_paths).apply(
    lambda df: list(df["function_code"])
)

In [19]:
functions_grouped.groupby(functions_grouped.index).agg("count")

1jsingh/rl_navigation agents/bst.py                                    1
1jsingh/rl_navigation agents/dqn_agent.py                              1
1jsingh/rl_navigation agents/model.py                                  1
5yearsKim/Conditional-Normalizing-Flow data/dataset.py                 1
5yearsKim/Conditional-Normalizing-Flow inference.py                    1
                                                                      ..
ztoString/CRNN_CTC_OCR_TensorFlow data/create_synth90k_tfrecord.py     1
ztoString/CRNN_CTC_OCR_TensorFlow tools/create_crnn_ctc_tfrecord.py    1
ztoString/CRNN_CTC_OCR_TensorFlow tools/eval_crnn_ctc.py               1
ztoString/CRNN_CTC_OCR_TensorFlow tools/inference_crnn_ctc.py          1
ztoString/CRNN_CTC_OCR_TensorFlow tools/train_crnn_ctc.py              1
Length: 24053, dtype: int64

In [20]:
sum(functions_grouped.iloc[:2], start=[])

['class Node(object):\n\n    def __init__(self, data):\n        self.data = data\n        self.freq = 1\n        self.leftchild = None\n        self.rightchild = None\n\n    def assign(self, another_node):\n        self.data = another_node.data\n        self.freq = another_node.freq\n        self.leftchild = another_node.leftchild\n        self.rightchild = another_node.rightchild\n\n    def remove_child(self, child_node):\n        assert ((child_node == self.leftchild) or (child_node == self.rightchild))\n        if (self.data < child_node.data):\n            self.rightchild = None\n        else:\n            self.leftchild = None',
 "class FixedSize_BinarySearchTree(object):\n\n    def __init__(self, capacity):\n        self.capacity = capacity\n        self.size = 0\n        self.values = deque(maxlen=capacity)\n        self.value_sum = 0\n        self.root = None\n\n    def update(self, value, idx, node=None):\n        '\\n\\t\\tupdate tree node value\\n\\t\\t'\n        assert (idx

In [21]:
function_paths = functions_df["repo_name"] + " " + functions_df["path"]

In [22]:
functions_grouped[function_paths.iloc[:5]]

trangvu/ape-npi translate/import_graph.py    [class ImportGraph():\n    '  Importing and ru...
trangvu/ape-npi translate/import_graph.py    [class ImportGraph():\n    '  Importing and ru...
trangvu/ape-npi translate/models.py          [def auto_reuse(fun):\n    "\n    Wrapper that...
trangvu/ape-npi translate/models.py          [def auto_reuse(fun):\n    "\n    Wrapper that...
trangvu/ape-npi translate/models.py          [def auto_reuse(fun):\n    "\n    Wrapper that...
dtype: object

In [23]:
repo_tasks = paperswithcode_with_imports_df[["repo", "tasks"]].groupby("repo").agg(sum)

In [24]:
def get_file_dataset(repo_tasks, functions_df):
    functions_df = functions_df.merge(repo_tasks, left_on="repo_name", right_on="repo")
    function_paths = functions_df["repo_name"] + " " + functions_df["path"]
    functions_groupby = functions_df.groupby(function_paths.values)
    functions_grouped = functions_groupby.apply(
        lambda df: frozenset(df["function_code"])
    )
    tasks_grouped = functions_groupby.apply(lambda df: list(df["tasks"]))
    data_df = (
        pd.DataFrame({"tasks": tasks_grouped, "functions": functions_grouped})
        .explode("tasks")
        .explode("tasks")
        .reset_index()
        .drop_duplicates()
    )
    data_df["functions"] = data_df["functions"].apply(list)
    return data_df

In [25]:
get_file_dataset(repo_tasks, functions_df).groupby("index").agg("count").sort_values(
    "functions"
)

,tasks,functions
index,,
1jsingh/rl_navigation agents/bst.py,1,1
geomstats/geomstats tests/tests_geomstats/test_normal.py,1,1
geomstats/geomstats tests/tests_geomstats/test_minkowski.py,1,1
geomstats/geomstats tests/tests_geomstats/test_mdm.py,1,1
geomstats/geomstats tests/tests_geomstats/test_matrices.py,1,1
...,...,...
kbardool/mrcnn3 mrcnn/ArchivedCode/model_fcn_02122019.py,13,13
kbardool/mrcnn3 mrcnn/ArchivedCode/model_fcn_09132018_.py,13,13
kbardool/mrcnn3 mrcnn/ArchivedCode/model_fcn_10142018_.py,13,13


In [26]:
def get_sample_indices_iterator(data_df, batch_size=8, shuffle=True):
    sample_index = data_df.index.values
    if shuffle:
        np.random.shuffle(sample_index)
    for i in range(0, len(sample_index), batch_size):
        yield sample_index[i : i + batch_size]

In [27]:
data_df = get_file_dataset(repo_tasks, functions_df)
sample_index = data_df.index.values
np.random.shuffle(sample_index)

In [28]:
data_df

,index,tasks,functions
112781,1jsingh/rl_navigation agents/bst.py,Atari Games,[class FixedSize_BinarySearchTree(object):\n\n...
9923,1jsingh/rl_navigation agents/dqn_agent.py,Atari Games,[class ReplayBuffer():\n\n def __init__(sel...
248440,1jsingh/rl_navigation agents/model.py,Atari Games,[class Qnetwork(nn.Module):\n\n def __init_...
65105,5yearsKim/Conditional-Normalizing-Flow data/da...,Colorization,[class ImgDatasets(torch.utils.data.Dataset):\...
65621,5yearsKim/Conditional-Normalizing-Flow data/da...,Image Generation,[class ImgDatasets(torch.utils.data.Dataset):\...
...,...,...,...
218204,ztoString/CRNN_CTC_OCR_TensorFlow tools/eval_c...,Scene Text Recognition,"[def _read_tfrecord(tfrecord_path, num_epochs=..."
24874,ztoString/CRNN_CTC_OCR_TensorFlow tools/infere...,Scene Text,"[def _int_to_string(value, char_map_dict=None)..."
112880,ztoString/CRNN_CTC_OCR_TensorFlow tools/infere...,Scene Text Recognition,"[def _int_to_string(value, char_map_dict=None)..."
108265,ztoString/CRNN_CTC_OCR_TensorFlow tools/train_...,Scene Text,"[def _read_tfrecord(tfrecord_path, num_epochs=..."


In [29]:
index_iter = get_sample_indices_iterator(data_df, batch_size=8)

In [34]:
def prepare_model_inputs(batch):
    indices = list(np.cumsum(batch["functions"].apply(len).values))
    file_function_idxs_start = np.array([0] + indices[:-1])
    file_function_idxs_end = indices
    reshaped_batch = batch.explode("functions")

    return (
        batch["tasks"].values,
        batch["index"].values,
        reshaped_batch["functions"].values,
        file_function_idxs_start,
        file_function_idxs_end,
    )

In [36]:
index_iter

<generator object get_sample_indices_iterator at 0x7f48bb10a2e0>

In [31]:
batch = data_df.iloc[next(iter(index_iter))]
(
    batch_tasks,
    batch_paths,
    batch_functions,
    file_function_idxs_start,
    file_function_idxs_end,
) = prepare_model_inputs(batch)

IndexError: positional indexers are out-of-bounds

In [ ]:
batch_functions

In [ ]:
file_function_idxs_start

In [ ]:
file_function_idxs_end

# Idea: use thinc

In [ ]:
def get_embeddings(model, tokenizer, texts, max_length=512):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, max_length=max_length)
    inputs["input_ids"] = inputs["input_ids"][:, :max_length]
    inputs["attention_mask"] = inputs["attention_mask"][:, :max_length]
    inputs = inputs.to(model.device)
    return model(**inputs).last_hidden_state.mean(axis=1)


# path_embeddings = get_embeddings(model, tokenizer, paths_batch)#.last_hidden_state.mean(axis=1)

In [ ]:
# function_embeddings = get_embeddings(model, tokenizer, functions_batch)

In [ ]:
# function_embeddings.dtype

In [ ]:
# functions_embeddings_reshaped = function_embeddings.repeat((5,1,1))

In [ ]:
# zrób dla jednego

In [ ]:
# first_file_functions = function_embeddings[file_function_idxs_start[0]:file_function_idxs_end[0]]

In [ ]:
# path_embeddings.shape
# first_file_functions

In [ ]:
class CrossAttentionSelector(nn.Module):
    def __init__(self):
        super(CrossAttentionSelector, self).__init__()

    def forward(
        self,
        path_embeddings,
        function_embeddings,
        file_function_idxs_start,
        file_function_idxs_end,
    ):
        attn = F.sigmoid(path_embeddings @ function_embeddings.T)
        attn_mask = torch.ones_like(attn).cuda().half()
        for i in range(len(file_function_idxs_start)):
            attn_mask[i, : file_function_idxs_start[i]] = 0
            attn_mask[i, file_function_idxs_end[i] :] = 0
        return (attn_mask * attn) @ function_embeddings

In [ ]:
selector = CrossAttentionSelector().cuda().half()

In [ ]:
class PathBasedFileEmbedder(nn.Module):
    def __init__(self, embedding_features, hidden_size, dropout_prob=0.5):
        super(PathBasedFileEmbedder, self).__init__()
        self.cross_attention_pooler = CrossAttentionSelector()
        self.path_dense = nn.Linear(embedding_features, hidden_size)
        self.function_dense = nn.Linear(embedding_features, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.bn3 = nn.BatchNorm1d(hidden_size)

    def forward(
        self,
        path_embeddings,
        function_embeddings,
        file_function_idxs_start,
        file_function_idxs_end,
    ):
        path_nl_embeddings = F.leaky_relu(self.bn1(self.path_dense(path_embeddings)))
        function_nl_embeddings = F.leaky_relu(
            self.bn2(self.function_dense(function_embeddings))
        )
        file_embeddings = self.cross_attention_pooler(
            path_nl_embeddings,
            function_nl_embeddings,
            file_function_idxs_start,
            file_function_idxs_end,
        )
        return (path_nl_embeddings, F.leaky_relu(self.bn3(file_embeddings)))

In [ ]:
file_embedder = PathBasedFileEmbedder(768, 20).cuda().half()

In [ ]:
path_embeddings = sentence_transformer_model.encode(
    batch_paths, convert_to_numpy=False, convert_to_tensor=True
).half()
function_embeddings = sentence_transformer_model.encode(
    batch_functions, convert_to_numpy=False, convert_to_tensor=True
).half()
task_embeddings = sentence_transformer_model.encode(
    batch_tasks, convert_to_numpy=False, convert_to_tensor=True
).half()

In [ ]:
selector(
    path_embeddings,
    function_embeddings,
    file_function_idxs_start,
    file_function_idxs_end,
)

In [ ]:
len(path_embeddings)

In [ ]:
class TripleSimilarityHead(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super(TripleSimilarityHead, self).__init__()
        self.input_size = input_size

    def forward(self, u, v, reference):
        # imilarity_features = torch.column_stack(
        #    [u, v, torch.abs(u-reference), torch.abs(v-reference)]
        # )
        # hidden = F.tanh(self.bn(self.dense(similarity_features)))
        return torch.column_stack(
            [
                torch.cosine_similarity(u, reference),
                torch.cosine_similarity(v, reference),
            ]
        )

In [ ]:
import sentence_transformers

In [ ]:
a = list(range(5))

a[2:]

In [ ]:
class AggregateSimilarityScorer(nn.Module):
    def __init__(
        self, sentence_transformer, embedding_features, hidden_size, dropout_prob=0.5
    ):
        super(AggregateSimilarityScorer, self).__init__()
        self.sentence_transformer = sentence_transformer
        self.file_embedder = PathBasedFileEmbedder(embedding_features, hidden_size)
        self.task_dense = nn.Linear(embedding_features, hidden_size)
        self.similarity_head = TripleSimilarityHead(hidden_size)
        self.bn_task = nn.BatchNorm1d(hidden_size)
        self.transformer_batch_size = 512

    def get_trainable_params(self):
        return itertools.chain(
            self.file_embedder.parameters(),
            self.task_dense.parameters(),
            self.similarity_head.parameters(),
            self.bn_task.parameters(),
        )

    def embed_tasks(self, tasks):
        with torch.no_grad():
            base_embeddings = self.sentence_transformer.encode(
                tasks, convert_to_numpy=False, convert_to_tensor=True
            ).half()
        task_embeddings = F.leaky_relu(self.bn_task(self.task_dense(base_embeddings)))
        return task_embeddings

    def forward(
        self,
        tasks,
        tasks_negative,
        paths,
        functions,
        file_function_idxs_start,
        file_function_idxs_end,
    ):
        all_inputs = list(tasks) + list(paths) + list(functions) + list(tasks_negative)
        embeddings = self.sentence_transformer.encode(
            all_inputs,
            convert_to_numpy=False,
            convert_to_tensor=True,
            batch_size=self.transformer_batch_size,
        ).half()
        base_task_embeddings = embeddings[: len(tasks)]
        base_path_embeddings = embeddings[len(tasks) : len(tasks) + len(paths)]
        function_embeddings = embeddings[len(tasks) + len(paths) : -len(tasks_negative)]
        base_negative_task_embeddings = embeddings[-len(tasks_negative) :]
        task_embeddings = F.leaky_relu(
            self.bn_task(self.task_dense(base_task_embeddings))
        )
        negative_task_embeddings = F.leaky_relu(
            self.bn_task(self.task_dense(base_negative_task_embeddings))
        )
        path_embeddings, file_embeddings = self.file_embedder(
            base_path_embeddings,
            function_embeddings,
            file_function_idxs_start,
            file_function_idxs_end,
        )
        positive_similarity = self.similarity_head(
            path_embeddings, file_embeddings, task_embeddings
        )
        negative_similarity = self.similarity_head(
            path_embeddings, file_embeddings, negative_task_embeddings
        )
        return positive_similarity, negative_similarity

    def embed_files(
        self, paths, functions, file_function_idxs_start, file_function_idxs_end
    ):
        with torch.no_grad():
            path_embeddings = self.sentence_transformer.encode(
                paths,
                convert_to_numpy=False,
                convert_to_tensor=True,
                batch_size=self.transformer_batch_size,
            ).half()
            function_embeddings = self.sentence_transformer.encode(
                functions,
                convert_to_numpy=False,
                convert_to_tensor=True,
                batch_size=self.transformer_batch_size,
            ).half()
            path_embeddings, file_embeddings = self.file_embedder(
                path_embeddings,
                function_embeddings,
                file_function_idxs_start,
                file_function_idxs_end,
            )
        return path_embeddings, file_embeddings

In [ ]:
embedding_features = 768

In [ ]:
# path_embeddings.shape#* functions_embeddings_reshaped

In [ ]:
sentence_transformer_model = sentence_transformer_model.half()

In [ ]:
similarity_scorer = (
    AggregateSimilarityScorer(
        sentence_transformer_model,
        embedding_features=embedding_features,
        hidden_size=512,
    )
    .cuda()
    .half()
)

In [ ]:
similarity_scorer

In [ ]:
tasks_batch = ["classification"] + ["domain adaptation"] * 4

base_path_embeddings = get_embeddings(model, tokenizer, paths_batch)
function_embeddings = get_embeddings(model, tokenizer, functions_batch)
base_task_embeddings = get_embeddings(model, tokenizer, tasks_batch)

In [ ]:
batch_paths.shape

In [ ]:
batch_tasks.shape

In [ ]:
random_tasks

In [ ]:
scores = similarity_scorer(
    batch_tasks,
    random_tasks,
    batch_paths,
    batch_functions,
    file_function_idxs_start,
    file_function_idxs_end,
)
# negative_scores = similarity_scorer(random_tasks, batch_paths, batch_functions, file_function_idxs_start, file_function_idxs_end)

In [ ]:
batch_size = 16

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("dark_background")

In [ ]:
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing=0.05):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.logsigmoid(x)
            minus_logprobs = torch.nn.functional.logsigmoid(-x)

            target_smoothed = target * (1 - self.smoothing) + self.smoothing / 2
            smoothed_loss = -logprobs * target_smoothed - minus_logprobs * (
                1 - target_smoothed
            )
            return smoothed_loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)

In [ ]:
label_smoothing = LabelSmoothing(0)

In [ ]:
log_probs = torch.tensor([-100, 100]).float().reshape(2, 1)  # ('int')

In [ ]:
targets = torch.tensor([0, 1]).reshape(2, 1)

In [ ]:
log_probs

In [ ]:
log_probs.shape

In [ ]:
targets.shape

In [ ]:
label_smoothing(log_probs, targets)

In [ ]:
import paperswithcode
import pandas as pd


def get_area_grouped_tasks():
    client = paperswithcode.PapersWithCodeClient()
    areas = client.area_list().results
    s = 0

    area_grouped_tasks = {}

    for a in areas:
        area_tasks = [
            t.id for t in client.area_task_list(a.id, items_per_page=1000).results
        ]
        area_grouped_tasks[a.id] = area_tasks
        n_tasks_per_area = len(area_tasks)
        print(a.id, ":", n_tasks_per_area)
        s += n_tasks_per_area
    tasks_df = pd.DataFrame(
        {"area": area_grouped_tasks.keys(), "task": area_grouped_tasks.values()}
    ).explode("task")
    tasks_df["task"] = tasks_df["task"].str.split("-").apply(" ".join)
    return tasks_df

In [ ]:
area_grouped_tasks = get_area_grouped_tasks()

In [ ]:
data_df["tasks"] = data_df["tasks"].str.lower()

In [ ]:
unique_tasks = data_df["tasks"].drop_duplicates()

In [ ]:
areas = [area_grouped_tasks[area_grouped_tasks["task"] == t] for t in unique_tasks]

In [ ]:
areas[3]

In [ ]:
def get_task_area(task):
    maybe_area = area_grouped_tasks[area_grouped_tasks["task"] == task]["area"]
    if len(maybe_area) == 0:
        return area_grouped_tasks["area"].sample(1).iloc[0]
    else:
        return maybe_area.iloc[0]

In [ ]:
def get_random_non_matching_task(task):
    task_area = get_task_area(task)
    return (
        area_grouped_tasks[area_grouped_tasks["area"] != task_area]
        .sample(1)
        .iloc[0]["task"]
    )

In [ ]:
get_random_non_matching_task("atari games")

In [ ]:
non_matching_tasks = dict(
    [(t, [get_random_non_matching_task(t) for __ in range(10)]) for t in unique_tasks]
)

In [ ]:
dict(non_matching_tasks)

In [ ]:
import random

In [ ]:
non_matching_tasks

In [ ]:
random.choice(non_matching_tasks["atari games"])

In [ ]:
similarity_scorer = (
    AggregateSimilarityScorer(
        sentence_transformer_model,
        embedding_features=embedding_features,
        hidden_size=512,
    )
    .cuda()
    .half()
)

In [ ]:
params = similarity_scorer.get_trainable_params()
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.5, nesterov=True)
batch_size = 32

In [ ]:
plt.style.use("dark_background")

In [ ]:
import livelossplot

In [ ]:
label_smoothing = LabelSmoothing(0.05)

In [ ]:
batch_size

In [ ]:
similarity_scorer = torch.load("output/similarity_scorer.pt")  # .state_dict()
similarity_scorer.transformer_batch_size = 512

In [ ]:
batch

In [ ]:
??prepare_model_inputs

In [ ]:
inputs = prepare_model_inputs(batch)
len(inputs)

In [ ]:
inputs[-2]

In [ ]:
path_embeddings, file_embeddings = similarity_scorer.embed_files(*inputs[1:])

In [ ]:
def get_batched_outputs(model, input, batch_size=256):
    outputs = []
    with torch.no_grad():
        for i in range(int(np.ceil(len(input) / batch_size))):
            batch_outputs = model(input[i * batch_size : (i + 1) * batch_size])
            outputs.append(batch_outputs.cpu().numpy())

    return np.row_stack(outputs)

In [ ]:
def prepare_path_file_features(similarity_scorer, data_df, batch_size=128):
    index_iterator = get_sample_indices_iterator(
        data_df, batch_size=batch_size, shuffle=False
    )
    all_path_embeddings = []
    all_file_embeddings = []
    with torch.no_grad():
        for indices in tqdm.tqdm(
            index_iterator, total=np.ceil(len(data_df) / batch_size)
        ):
            batch = data_df.iloc[indices]
            (
                __,
                batch_paths,
                batch_functions,
                file_function_idxs_start,
                file_function_idxs_end,
            ) = prepare_model_inputs(batch)
            path_embeddings, file_embeddings = similarity_scorer.embed_files(
                batch_paths,
                batch_functions,
                file_function_idxs_start,
                file_function_idxs_end,
            )
            all_path_embeddings.append(path_embeddings.cpu().numpy())
            all_file_embeddings.append(file_embeddings.cpu().numpy())
    return np.row_stack(all_path_embeddings), np.row_stack(all_file_embeddings)

In [ ]:
sample_data_df["tasks"] = sample_data_df["tasks"].str.lower()

In [ ]:
sample_data_df = data_df.iloc[:1000].reset_index(drop=True)
from sklearn import metrics

task_embeddings = get_batched_outputs(
    similarity_scorer.embed_tasks, area_grouped_tasks["task"].values
)

matching_task_embeddings = get_batched_outputs(
    similarity_scorer.embed_tasks, sample_data_df["tasks"].values
)

In [ ]:
path_features, file_features = prepare_path_file_features(
    similarity_scorer, sample_data_df, batch_size=128
)

In [ ]:
matching_path_task_similarity = np.diag(
    metrics.pairwise.cosine_similarity(path_features, matching_task_embeddings)
)
matching_file_task_similarity = np.diag(
    metrics.pairwise.cosine_similarity(file_features, matching_task_embeddings)
)

path_task_similarity = metrics.pairwise.cosine_similarity(
    path_features, task_embeddings
)
file_task_similarity = metrics.pairwise.cosine_similarity(
    file_features, task_embeddings
)

In [ ]:
sample_data_df

In [ ]:
pd.Series(matching_path_task_similarity).describe()

In [ ]:
pd.Series(path_task_similarity.reshape(-1)).describe()

In [ ]:
pd.Series(matching_file_task_similarity + matching_path_task_similarity).describe()

In [ ]:
pd.Series((path_task_similarity + file_task_similarity).reshape(-1)).describe()

In [ ]:
path_task_similarity.shape

In [ ]:
pd.Series(
    (path_task_similarity / matching_path_task_similarity[:, np.newaxis]).reshape(-1)
).describe()

In [ ]:
pd.Series(
    (file_task_similarity / matching_file_task_similarity[:, np.newaxis]).reshape(-1)
).describe()

In [ ]:
sns.histplot(matching_path_task_similarity.reshape(-1), label="matching")
sns.histplot(path_task_similarity[:30].reshape(-1), label="other", color="red")
plt.legend()

In [ ]:
sns.histplot(matching_file_task_similarity.reshape(-1), label="matching", color="red")
plt.show()
sns.histplot(file_task_similarity.reshape(-1), label="other", alpha=0.5)
plt.legend()

In [ ]:
sns.histplot(
    (matching_path_task_similarity + matching_file_task_similarity),
    label="matching",
    color="red",
)
plt.show()
sns.histplot(
    (path_task_similarity + file_task_similarity).reshape(-1), label="other", alpha=0.5
)
plt.legend()

In [ ]:
task_embeddings

In [ ]:
# Dla pojedynczego negatywnego taska problem jest za łatwy i się nie generalizuje - trzeba zrobić jakiś ranking czy coś

In [ ]:
plotlosses = livelossplot.PlotLosses(from_step=1)

for epoch in range(1):
    print("epoch", epoch)
    index_iterator = get_sample_indices_iterator(data_df, batch_size=batch_size)
    non_matching_tasks = dict(
        [
            (t, [get_random_non_matching_task(t) for __ in range(10)])
            for t in unique_tasks
        ]
    )
    n_batches = np.ceil(len(data_df) / batch_size)
    for i, batch_indices in tqdm.tqdm(enumerate(index_iterator), total=n_batches):
        batch = data_df.iloc[batch_indices]
        (
            batch_tasks,
            batch_paths,
            batch_functions,
            file_function_idxs_start,
            file_function_idxs_end,
        ) = prepare_model_inputs(batch)
        random_tasks = [random.choice(non_matching_tasks[t]) for t in batch_tasks]
        scores, negative_scores = similarity_scorer(
            batch_tasks,
            random_tasks,
            batch_paths,
            batch_functions,
            file_function_idxs_start,
            file_function_idxs_end,
        )
        scores = scores.sum(dim=-1, keepdim=True)
        negative_scores = negative_scores.sum(dim=-1, keepdim=True)
        all_scores = torch.cat([scores, negative_scores])
        label = torch.ones(len(batch) * 2, 1).cuda().half()
        label[batch_size:] = 0

        loss = label_smoothing(all_scores, label)
        loss.backward()
        optimizer.step()
        acc = (
            ((scores > negative_scores)).float().mean().item()
        )  # detach().cpu().numpy().mean()
        plotlosses.update({"loss": loss.item(), "accuracy": acc})
        if i % 5 == 0:
            plotlosses.send()

In [ ]:
!ls 

In [ ]:
!ls o

In [ ]:
# torch.save(similarity_scorer, "output/similarity_scorer.pt")#.state_dict()

In [ ]:
random_tasks = (
    repo_tasks.sample(10).explode("tasks").sample(len(batch_tasks))["tasks"].values
)

In [ ]:
batch_tasks

In [ ]:
random_tasks

In [ ]:
import thinc

batch size mismatch

In [ ]:
len(functions_batch)

In [ ]:
functions_batch

In [ ]:
model.b

In [ ]:
function_embeddings = get_embedding(model, tokenizer, functions_batch).last_hidden_state

In [ ]:
function_embeddings.shape

In [ ]:
function_embeddings.shape

In [ ]:
torch.nn.Sigmoid()(embeddings)